In [1]:
!pip install keras

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


In [2]:
import pandas as pd

In [3]:
test=pd.read_csv("test_all.csv")

In [4]:
item=test["user_items"].values.tolist()

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
docs = item

# 只考虑最常见的8个单词
max_words = 3000
# 统一的序列化长度
# 大于这个长度截断，小于这个长度用0补全
maxlen = 10
# 嵌入的维度
embedding_dim = 128

# 分词
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(docs)
# 字典
word_index = tokenizer.word_index

# 序列化
sequences = tokenizer.texts_to_sequences(docs)
# 统一序列长度
data = pad_sequences(sequences, maxlen=maxlen)

# Embedding模型
model = Sequential()
# Embedding至少需要max_wrods和embedding_dim两个参数
model.add(Embedding(max_words, embedding_dim, input_length=maxlen, name='embedding'))
model.compile('rmsprop', 'mse')

out = model.predict(data)
# print(out)
# print(out.shape)
# # 查看权重
# layer = model.get_layer('embedding')
# print(layer.get_weights())

embedding_list=[]
for i in out:
    temp=sum(i)/10
    temp=temp.tolist()
    embedding_list.append(temp)

Using TensorFlow backend.


In [6]:
pip install lightgbm

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [7]:
import lightgbm as lgb
# 模型加载
gbm = lgb.Booster(model_file='age_model.txt')
# 模型预测
age_pred = gbm.predict(embedding_list)
age_pred=[list(x).index(max(x)) for x in age_pred]

In [8]:
import lightgbm as lgb
# 模型加载
gbm = lgb.Booster(model_file='gender_model.txt')
# 模型预测
gender_pred = gbm.predict(embedding_list, num_iteration=gbm.best_iteration)
gender_pred=[list(x).index(max(x)) for x in gender_pred]

In [9]:
submission=pd.DataFrame(test["user_id"])

In [10]:
submission["predicted_age"]=age_pred

In [11]:
submission["predicted_gender"]=gender_pred

In [12]:
submission.to_csv("submission_5 25.csv",index=False)

In [13]:
submission

,user_id,predicted_age,predicted_gender
0,3000001,6,1
1,3000002,2,1
2,3000003,3,1
3,3000004,2,1
4,3000005,3,1
...,...,...,...
999995,3999996,2,1
999996,3999997,3,1
999997,3999998,2,2
999998,3999999,2,1
